# Simple RAG (Retrieval-Augmented Generation) System for CSV Files

## Overview

This code implements a basic Retrieval-Augmented Generation (RAG) system for processing and querying CSV documents. The system encodes the document content into a vector store, which can then be queried to retrieve relevant information.

# CSV File Structure and Use Case
The CSV file contains dummy customer data, comprising various attributes like first name, last name, company, etc. This dataset will be utilized for a RAG use case, facilitating the creation of a customer information Q&A system.

## Key Components

1. Loading and spliting csv files.
2. Vector store creation using [FAISS](https://engineering.fb.com/2017/03/29/data-infrastructure/faiss-a-library-for-efficient-similarity-search/) and OpenAI embeddings
3. Retriever setup for querying the processed documents
4. Creating a question and answer over the csv data.

## Method Details

### Document Preprocessing

1. The csv is loaded using langchain Csvloader
2. The data is split into chunks.


### Vector Store Creation

1. OpenAI embeddings are used to create vector representations of the text chunks.
2. A FAISS vector store is created from these embeddings for efficient similarity search.

### Retriever Setup

1. A retriever is configured to fetch the most relevant chunks for a given query.

## Benefits of this Approach

1. Scalability: Can handle large documents by processing them in chunks.
2. Flexibility: Easy to adjust parameters like chunk size and number of retrieved results.
3. Efficiency: Utilizes FAISS for fast similarity search in high-dimensional spaces.
4. Integration with Advanced NLP: Uses OpenAI embeddings for state-of-the-art text representation.

## Introduction to LangChain

This notebook, while demonstrating a RAG system, also serves as a practical introduction to [LangChain](https://www.langchain.com/). LangChain is a powerful open-source framework designed to simplify the creation of applications using large language models (LLMs). It provides a standard interface for chains, lots of integrations with other tools, and end-to-end chains for common applications.

LangChain can be seen as a higher-level library that orchestrates the different components of a RAG system (and other LLM applications) in a more streamlined and modular way. Instead of writing boilerplate code to connect your data source, embedding model, vector store, and LLM, LangChain provides convenient abstractions to do so.

In this notebook, we will be using several LangChain components, which will be explained in more detail in the following sections.

# Package Installation and Imports

The cell below installs all necessary packages required to run this notebook.


In [ ]:
# Install required packages
!pip install faiss-cpu langchain langchain-community langchain-openai pandas python-dotenv

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from pathlib import Path
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
import os
from dotenv import load_dotenv

# Load environment variables from a .env file
env_path = "/content/drive/MyDrive/.env"
load_dotenv(env_path)

# Set the OpenAI API key environment variable
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [ ]:
# Add Langchain create_agent_chain (so after you use it in 3.2 and 4.2)

# CSV File Structure and Use Case
The CSV file contains dummy customer data, comprising various attributes like first name, last name, company, etc. This dataset will be utilized for a RAG use case, facilitating the creation of a customer information Q&A system.

In [1]:
# Download required data files
import os
os.makedirs('data', exist_ok=True)

# Download the PDF document used in this notebook
!wget -O data/Understanding_Climate_Change.pdf https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/Understanding_Climate_Change.pdf
!wget -O data/customers-100.csv https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/customers-100.csv


--2026-01-22 18:04:47--  https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/Understanding_Climate_Change.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 206372 (202K) [application/octet-stream]
Saving to: ‘data/Understanding_Climate_Change.pdf’

data/Understanding_ 100%[===================>] 201.54K  --.-KB/s    in 0.02s   

2026-01-22 18:04:47 (7.89 MB/s) - ‘data/Understanding_Climate_Change.pdf’ saved [206372/206372]

--2026-01-22 18:04:47--  https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/customers-100.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:4

In [ ]:
import pandas as pd

file_path = ('data/customers-100.csv') # insert the path of the csv file
data = pd.read_csv(file_path)

#preview the csv file
data.head()

### Loading Data with LangChain's `CSVLoader`

The first step in our RAG pipeline is to load the data from our CSV file. LangChain provides a variety of document loaders for different file formats, and in this case, we use the `CSVLoader`. This loader reads the CSV file and creates a `Document` object for each row. Each document contains the page content (the row data) and metadata.

In [ ]:
loader = CSVLoader(file_path=file_path)
docs = loader.load_and_split()

### Embeddings with LangChain and OpenAI

After loading the data, we need to create vector embeddings for our documents. LangChain provides a seamless integration with various embedding models, including OpenAI's. The `OpenAIEmbeddings` class is a wrapper around the OpenAI API that allows us to easily generate embeddings for our text data. These embeddings are high-dimensional vectors that capture the semantic meaning of the text, which is crucial for the retrieval step in our RAG system.

In [ ]:
embeddings = OpenAIEmbeddings()
index = faiss.IndexFlatL2(len(OpenAIEmbeddings().embed_query(" ")))

### Vector Stores in LangChain: FAISS

Once we have the embeddings, we need a place to store them and efficiently search for similar vectors. This is where vector stores come in. LangChain supports a wide range of vector stores, and in this notebook, we use `FAISS` (Facebook AI Similarity Search), a library for efficient similarity search and clustering of dense vectors.

LangChain's `FAISS` class provides a convenient wrapper around the FAISS library, allowing us to create a vector store from our documents and embeddings with just a few lines of code. This vector store will be used by our retriever to find the most relevant documents for a given query.

In [ ]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

vector_store = FAISS(
    embedding_function=OpenAIEmbeddings(),
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

Add the splitted csv data to the vector store

In [ ]:
vector_store.add_documents(documents=docs)

### LangChain Chains: `create_retrieval_chain`

The core of LangChain lies in its concept of 'chains'. Chains allow us to combine multiple components together to create a single, coherent application. In our case, we want to create a retrieval-based question-answering chain.

The `create_retrieval_chain` function is a helper function that simplifies the process of creating such a chain. It takes a retriever and a language model as input and creates a chain that first retrieves relevant documents from the vector store and then passes them to the language model to generate an answer. This is the essence of the Retrieval-Augmented Generation (RAG) pattern.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

retriever = __.as_retriever()

# Set up system prompt
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", __), ## This is the context
    ("human", "{input}"), ## This will be our question

])

# Create the question-answer chain
question_answer_chain = create_stuff_documents_chain(__, __) ## Defines the structure of the prompt that will generate the answer
rag_chain = create_retrieval_chain(__, __) ## Builds the chain of retrieve + generate answer

Query the rag bot with a question based on the CSV data

In [ ]:
answer= __({"__": "which company does sheryl Baxter work for?"})
answer['answer']

## **LAB3: RAG Q&A Bot**
<u>**Download the following files to create your own RAG system:**</u>

In [ ]:
!wget -O app.py https://raw.githubusercontent.com/Bor90/Nuclio_GenAI/refs/heads/main/Labs/RAG_QandA/app.py
!wget -O requirements.txt https://raw.githubusercontent.com/Bor90/Nuclio_GenAI/refs/heads/main/Labs/RAG_QandA/requirements.txt

## Conclusion

This simple RAG system provides a solid foundation for building more complex information retrieval and question-answering systems. By encoding document content into a searchable vector store, it enables efficient retrieval of relevant information in response to queries. This approach is particularly useful for applications requiring quick access to specific information within a csv file.

![](https://europe-west1-rag-techniques-views-tracker.cloudfunctions.net/rag-techniques-tracker?notebook=all-rag-techniques--simple-csv-rag)